## Building A Chatbot
In this video We'll go over an example of how to design and implement an LLM-powered chatbot. This chatbot will be able to have a conversation and remember previous interactions.

Note that this chatbot that we build will only use the language model to have a conversation. There are several other related concepts that you may be looking for:

- Conversational RAG: Enable a chatbot experience over an external source of data
- Agents: Build a chatbot that can take actions

This video tutorial will cover the basics which will be helpful for those two more advanced topics.

In [ ]:
import os
from dotenv import load_dotenv
load_dotenv() ## aloading all the environment variable

groq_api_key=os.getenv("GROQ_API_KEY")


'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'

In [3]:
from langchain_groq import ChatGroq
model=ChatGroq(model="Gemma2-9b-It",groq_api_key=groq_api_key)
model

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x000001774EA0BFE0>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x000001774E941430>, model_name='Gemma2-9b-It', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [4]:
from langchain_core.messages import HumanMessage
model.invoke([HumanMessage(content="Hi , My name is Sagnik and I am a Data Scientist at CVS Health")])

AIMessage(content="Hello Sagnik, it's nice to meet you!\n\nThat's interesting, being a Data Scientist at CVS Health must be quite rewarding. What kind of projects are you working on?  Do you have any particular areas of focus within data science?  \n\nI'm always eager to learn more about how data science is being applied in different industries.  \n\n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 78, 'prompt_tokens': 25, 'total_tokens': 103, 'completion_time': 0.141818182, 'prompt_time': 0.002093661, 'queue_time': 0.066224948, 'total_time': 0.143911843}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run--8d069594-b5bb-408a-816b-718be6707c64-0', usage_metadata={'input_tokens': 25, 'output_tokens': 78, 'total_tokens': 103})

In [5]:
from langchain_core.messages import AIMessage
model.invoke(
    [
        HumanMessage(content="Hi , My name is Sagnik and I am a Data Scientist at CVS Health"),
        AIMessage(content="Hello Sagnik! It's nice to meet you. \n\nAs a Data Scientist, what kind of projects are you working on these days? \n\nI'm always eager to learn more about the exciting work being done in the field of AI.\n"),
        HumanMessage(content="Hey What's my name and what do I do?")
    ]
)

AIMessage(content="You said your name is Sagnik, and you are a Data Scientist at CVS Health.  😊 \n\nIs there anything else you'd like to tell me about yourself or your work? I'm ready to listen!  \n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 50, 'prompt_tokens': 100, 'total_tokens': 150, 'completion_time': 0.090909091, 'prompt_time': 0.006562068, 'queue_time': 0.08538710399999999, 'total_time': 0.097471159}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run--2abb0a23-4f38-4f44-bd3b-5ac24439a6f7-0', usage_metadata={'input_tokens': 100, 'output_tokens': 50, 'total_tokens': 150})

### Message History
We can use a Message History class to wrap our model and make it stateful. This will keep track of inputs and outputs of the model, and store them in some datastore. Future interactions will then load those messages and pass them into the chain as part of the input. Let's see how to use this!

In [ ]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

store={}
# Must be in a key value pair fashion
config={"configurable":{"session_id":"chat1"}}

def get_session_history(session_id:str)->BaseChatMessageHistory:
    if session_id not in store:
        store[session_id]=ChatMessageHistory()
    return store[session_id]

with_message_history=RunnableWithMessageHistory(model,get_session_history)

In [11]:
response=with_message_history.invoke(
    [HumanMessage(content="Hi , My name is Sagnik and I am a Data Scientist")],
    config=config
)
response.content

"Hi Sagnik! \n\nThat's awesome!  What kind of data science work do you enjoy doing most?  Do you have a favorite project you've worked on?  😄 \n\n"

In [12]:
with_message_history.invoke(
    [HumanMessage(content="What's my name?")],
    config=config,
)

AIMessage(content="Your name is Sagnik!  😊  \n\nI remembered from our earlier conversation.  It's nice to meet you, Sagnik.  What can I do for you today?\n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 41, 'prompt_tokens': 148, 'total_tokens': 189, 'completion_time': 0.074545455, 'prompt_time': 0.007065259, 'queue_time': 0.109468629, 'total_time': 0.081610714}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run--f7d488e3-a885-43cc-b05e-6c1a8e256c0c-0', usage_metadata={'input_tokens': 148, 'output_tokens': 41, 'total_tokens': 189})

In [13]:
## change the config-->session id
config1={"configurable":{"session_id":"chat2"}}
response=with_message_history.invoke(
    [HumanMessage(content="Whats my name")],
    config=config1
)
response.content

"As an AI, I have no memory of past conversations and don't know your name. Please tell me your name! 😊\n"

In [14]:
response=with_message_history.invoke(
    [HumanMessage(content="Hey My name is John")],
    config=config1
)
response.content

"Hi John! It's nice to meet you. 👋 \n\nWhat can I do for you today?\n"

In [15]:
response=with_message_history.invoke(
    [HumanMessage(content="Whats my name")],
    config=config1
)
response.content

'Your name is John.  😄  \n\nIs there anything else I can help you with?\n'

### Prompt templates
Prompt Templates help to turn raw user information into a format that the LLM can work with. In this case, the raw user input is just a message, which we are passing to the LLM. Let's now make that a bit more complicated. First, let's add in a system message with some custom instructions (but still taking messages as input). Next, we'll add in more input besides just the messages.

In [19]:
from langchain_core.prompts import ChatPromptTemplate,MessagesPlaceholder
prompt=ChatPromptTemplate.from_messages(
    [
        ("system","You are a helpful assistant.Amnswer all the question to the nest of your ability"),
        MessagesPlaceholder(variable_name="messages")
    ]
)

chain=prompt|model

In [20]:
chain.invoke({"messages":[HumanMessage(content="Hi My name is Sagnik Rana")]})

AIMessage(content="Hello Sagnik Rana, it's nice to meet you!\n\nI'm ready to answer your questions to the best of my ability.  \n\nWhat can I help you with today? 😄  \n\n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 45, 'prompt_tokens': 33, 'total_tokens': 78, 'completion_time': 0.081818182, 'prompt_time': 0.0024011, 'queue_time': 0.09044118999999999, 'total_time': 0.084219282}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run--bf1540ac-abac-44ba-98d3-d528ff0cd405-0', usage_metadata={'input_tokens': 33, 'output_tokens': 45, 'total_tokens': 78})

In [21]:
with_message_history=RunnableWithMessageHistory(chain,get_session_history)
with_message_history

RunnableWithMessageHistory(bound=RunnableBinding(bound=RunnableBinding(bound=RunnableLambda(_enter_history), kwargs={}, config={'run_name': 'load_history'}, config_factories=[])
| RunnableBinding(bound=RunnableLambda(_call_runnable_sync), kwargs={}, config={'run_name': 'check_sync_or_async'}, config_factories=[]), kwargs={}, config={'run_name': 'RunnableWithMessageHistory'}, config_factories=[]), kwargs={}, config={}, config_factories=[], get_session_history=<function get_session_history at 0x000001774EBC5B20>, history_factory_config=[ConfigurableFieldSpec(id='session_id', annotation=<class 'str'>, name='Session ID', description='Unique identifier for a session.', default='', is_shared=True, dependencies=None)])

In [25]:
config = {"configurable": {"session_id": "chat3"}}
response=with_message_history.invoke(
    [HumanMessage(content="Hi My name is Sagnik")],
    config=config
)

response

AIMessage(content="Hi Sagnik, it's nice to meet you! 👋 \n\nIs there anything I can help you with today? 😊  \n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 30, 'prompt_tokens': 131, 'total_tokens': 161, 'completion_time': 0.054545455, 'prompt_time': 0.006731369, 'queue_time': 0.109104693, 'total_time': 0.061276824}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run--d2f24e26-f119-438d-89d3-2d629b49cfaa-0', usage_metadata={'input_tokens': 131, 'output_tokens': 30, 'total_tokens': 161})

In [26]:
response = with_message_history.invoke(
    [HumanMessage(content="What's my name?")],
    config=config,
)

response.content

'Your name is Sagnik!  I remember. 😊  \n\nDo you have any other questions for me?  \n\n\n\n'

In [27]:
## Add more complexity

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. Answer all questions to the best of your ability in {language}.",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

chain = prompt | model

In [37]:
response=chain.invoke({"messages":[HumanMessage(content="Hi My name is Sagnik")],"language":"Russian"})
response.content

'Привет, Сагни!  \n\nРад познакомиться. Чем могу тебе помочь? 😄 \n'

Let's now wrap this more complicated chain in a Message History class. This time, because there are multiple keys in the input, we need to specify the correct key to use to save the chat history.

In [29]:
with_message_history=RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="messages"
)

In [39]:
config = {"configurable": {"session_id": "chat4"}}
repsonse=with_message_history.invoke(
    {'messages': [HumanMessage(content="Hi,I am Sagnik")],"language":"Russian"},
    config=config
)
repsonse.content

'Привет, Сагник! 😊 \n\nПриятно познакомиться. Что ты хочешь узнать или сделать? \n'

In [40]:
response = with_message_history.invoke(
    {"messages": [HumanMessage(content="whats my name?")], "language": "Russian"},
    config=config,
)
response.content

'Твое имя Сагник. 😊 \n'

### Managing the Conversation History
One important concept to understand when building chatbots is how to manage conversation history. If left unmanaged, the list of messages will grow unbounded and potentially overflow the context window of the LLM. Therefore, it is important to add a step that limits the size of the messages you are passing in.
'trim_messages' helper to reduce how many messages we're sending to the model. The trimmer allows us to specify how many tokens we want to keep, along with other parameters like if we want to always keep the system message and whether to allow partial messages

In [42]:
from langchain_core.messages import SystemMessage,trim_messages
trimmer=trim_messages(
    max_tokens=45,
    strategy="last",
    token_counter=model,
    include_system=True,
    allow_partial=False,
    start_on="human"
)
messages = [
    SystemMessage(content="you're a good assistant"),
    HumanMessage(content="hi! I'm bob"),
    AIMessage(content="hi!"),
    HumanMessage(content="I like vanilla ice cream"),
    AIMessage(content="nice"),
    HumanMessage(content="whats 2 + 2"),
    AIMessage(content="4"),
    HumanMessage(content="thanks"),
    AIMessage(content="no problem!"),
    HumanMessage(content="having fun?"),
    AIMessage(content="yes!"),
]
trimmer.invoke(messages)

c:\Users\sagnik\anaconda3\envs\GenAI\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
None of PyTorch, TensorFlow >= 2.0, or Flax have been found. Models won't be available and only tokenizers, configuration and file/data utilities can be used.
c:\Users\sagnik\anaconda3\envs\GenAI\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\sagnik\.cache\huggingface\hub\models--gpt2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitat

[SystemMessage(content="you're a good assistant", additional_kwargs={}, response_metadata={}),
 HumanMessage(content='I like vanilla ice cream', additional_kwargs={}, response_metadata={}),
 AIMessage(content='nice', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='whats 2 + 2', additional_kwargs={}, response_metadata={}),
 AIMessage(content='4', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='thanks', additional_kwargs={}, response_metadata={}),
 AIMessage(content='no problem!', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='having fun?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='yes!', additional_kwargs={}, response_metadata={})]

In [43]:
from operator import itemgetter

from langchain_core.runnables import RunnablePassthrough
# Itemgetter will help us to get all the messages that are available in the Prompt Template.
chain=(
    RunnablePassthrough.assign(messages=itemgetter("messages")|trimmer)
    | prompt
    | model
)

response=chain.invoke(
    {
    "messages":messages + [HumanMessage(content="What ice cream do i like")],
    "language":"English"
    }
)
response.content

"As a large language model, I don't have access to your personal preferences like your favorite ice cream flavor.  \n\nDo you want to tell me what your favorite flavor is? 😊🍦\n"

In [44]:
response = chain.invoke(
    {
        "messages": messages + [HumanMessage(content="what math problem did i ask")],
        "language": "English",
    }
)
response.content

'You asked "What is 2 + 2?" 😊  \n\n\n\nLet me know if you have any other questions!\n'

In [45]:
## Lets wrap this in the MEssage History
with_message_history = RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="messages",
)
config={"configurable":{"session_id":"chat5"}}

In [46]:
response = with_message_history.invoke(
    {
        "messages": messages + [HumanMessage(content="whats my name?")],
        "language": "English",
    },
    config=config,
)

response.content

"As an AI, I don't have access to past conversations or any personal information about you, including your name. 😊\n\nIf you'd like to tell me your name, I'd love to know!\n"

In [47]:
response = with_message_history.invoke(
    {
        "messages": [HumanMessage(content="what math problem did i ask?")],
        "language": "English",
    },
    config=config,
)

response.content

"As a helpful and friendly AI assistant, I have no memory of past conversations.\n\nIf you'd like to ask me a math problem, I'm happy to help! 😊  Just let me know what it is. \n\n"